In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geopandas as gpd
import pymannkendall as mk
import numpy as np
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
from scipy.stats import spearmanr
import geopandas as gpd
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from shapely.geometry import Polygon, MultiPolygon
import matplotlib.patches as patches
import matplotlib.lines as mlines

In [ ]:
df = pd.read_csv('GWS.csv')

In [ ]:

def trend_test(row):
    lat, lon = row['lat'], row['lon']
    time_series = row.iloc[2:]
    
    # Performed the Seasonal Mann-Kendall test on the time series
    result = mk.seasonal_test(time_series, period=12)
    
    # Calculated the mean annual trend using Sen's slope
    annual_trend = mk.sens_slope(time_series.iloc[::12])  # Select only annual data points
    
    # Extracted the p-value, Kendall's tau, and mean annual trend from the test result
    p_value = result.p
    tau = result.Tau
    sen_slope = annual_trend.slope
    return pd.Series([lat, lon, p_value, tau, sen_slope])

# Applied the trend_test function on each row of the DataFrame
result_df = df.apply(trend_test, axis=1)

# Renamed the columns of the result DataFrame
result_df.columns = ['lat', 'lon', 'p_value', 'tau', 'sen_slope']

print(result_df)

In [ ]:
result_df.to_csv('Senlope_recharge.csv', index=None)

In [ ]:
A, B = result_df['sen_slope'].min(), result_df['sen_slope'].max()
A, B

In [ ]:
# Function to add a scalebar
def add_scalebar(ax, length, location=(0.5, 0.05), linewidth=4):
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]
    tmc = ccrs.TransverseMercator(sbllx, sblly)
    x0, x1, y0, y1 = ax.get_extent(tmc)
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]
    bar_xs = [sbx - length * 500, sbx + length * 500]
    ax.plot(bar_xs, [sby, sby], transform=tmc, color='k', linewidth=linewidth)
    ax.text(sbx, sby, str(length) + ' km', transform=tmc,
            horizontalalignment='center', verticalalignment='bottom')

min_value = A
max_value = B

# Got the absolute maximum value in your data
abs_max = max(abs(min_value), abs(max_value))

# Redefined min_value and max_value to be symmetric around zero
min_value = -abs_max
max_value = abs_max

interval = (max_value - min_value) / 10
bounds = np.linspace(min_value, max_value, 11)
neg_cmap = mcolors.LinearSegmentedColormap.from_list("neg_cmap", ["darkred", "yellow"], 5)
pos_cmap = mcolors.LinearSegmentedColormap.from_list("pos_cmap", ["lightblue", "darkblue"], 5)
cmap = mcolors.ListedColormap(np.concatenate((neg_cmap(np.linspace(0, 1, 5)), pos_cmap(np.linspace(0, 1, 5)))))
norm = mcolors.BoundaryNorm(bounds, cmap.N)

def get_color(value, bounds, colors):
    for i in range(len(bounds) - 1):
        if value >= bounds[i] and value < bounds[i + 1]:
            return colors[i]
    return colors[-1]

result_df['color'] = result_df['sen_slope'].apply(lambda x: get_color(x, bounds, cmap.colors))

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

gdf = gpd.read_file('study_extent.shp')

for geometry in gdf['geometry']:
    if isinstance(geometry, Polygon):
        x, y = geometry.exterior.coords.xy
        ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))
    elif isinstance(geometry, MultiPolygon):
        for subgeometry in geometry:
            x, y = subgeometry.exterior.coords.xy
            ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=2)
ax.gridlines(draw_labels=True, linestyle='--')

sc = ax.scatter(result_df['lon'], result_df['lat'], c=result_df['color'],
                marker='s', s=30, edgecolor=result_df['color'], transform=ccrs.PlateCarree())

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

cbar = plt.colorbar(sm, orientation='horizontal', shrink=0.5, ticks=bounds, pad=0.05)
cbar.set_label('Mean Annual GWS trend (mm)', fontsize=15)
cbar.ax.set_xticklabels(['{:.2f}'.format(b) for b in bounds])

ax.set_extent([-10, 49, 25.5, 49])


add_scalebar(ax, 500)

arrow_x, arrow_y = 0.97, .9
ax.text(arrow_x, arrow_y, u'\u25B2\nN', transform=ax.transAxes, ha='center', va='bottom', fontsize=32, fontweight='bold')

plt.title('Annual Trend in GWS (GWS-without soil moisture)', fontsize=20, pad=20)
plt.show()


In [ ]:
df = pd.read_csv('recharge_with_coord.csv')

In [ ]:

def trend_test(row):
    lat, lon = row['lat'], row['lon']
    time_series = row.iloc[2:]
    
    # Performed the Seasonal Mann-Kendall test on the time series
    result = mk.seasonal_test(time_series, period=12)
    
    # Calculated the mean annual trend using Sen's slope
    annual_trend = mk.sens_slope(time_series.iloc[::12])  # Select only annual data points
    
    # Extracted the p-value, Kendall's tau, and mean annual trend from the test result
    p_value = result.p
    tau = result.Tau
    sen_slope = annual_trend.slope
    return pd.Series([lat, lon, p_value, tau, sen_slope])

# Applied the trend_test function on each row of the DataFrame
result_df = df.apply(trend_test, axis=1)

# Renamed the columns of the result DataFrame
result_df.columns = ['lat', 'lon', 'p_value', 'tau', 'sen_slope']

print(result_df)

In [ ]:
result_df.to_csv('Senlope_recharge.csv', index=None)

In [ ]:
A, B = result_df['sen_slope'].min(), result_df['sen_slope'].max()
A, B

In [ ]:
# Function to add a scalebar
def add_scalebar(ax, length, location=(0.5, 0.05), linewidth=4):
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]
    tmc = ccrs.TransverseMercator(sbllx, sblly)
    x0, x1, y0, y1 = ax.get_extent(tmc)
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]
    bar_xs = [sbx - length * 500, sbx + length * 500]
    ax.plot(bar_xs, [sby, sby], transform=tmc, color='k', linewidth=linewidth)
    ax.text(sbx, sby, str(length) + ' km', transform=tmc,
            horizontalalignment='center', verticalalignment='bottom')

min_value = A
max_value = B

# Got the absolute maximum value in your data
abs_max = max(abs(min_value), abs(max_value))

# Redefined min_value and max_value to be symmetric around zero
min_value = -abs_max
max_value = abs_max

interval = (max_value - min_value) / 10
bounds = np.linspace(min_value, max_value, 11)
neg_cmap = mcolors.LinearSegmentedColormap.from_list("neg_cmap", ["darkred", "yellow"], 5)
pos_cmap = mcolors.LinearSegmentedColormap.from_list("pos_cmap", ["lightblue", "darkblue"], 5)
cmap = mcolors.ListedColormap(np.concatenate((neg_cmap(np.linspace(0, 1, 5)), pos_cmap(np.linspace(0, 1, 5)))))
norm = mcolors.BoundaryNorm(bounds, cmap.N)

def get_color(value, bounds, colors):
    for i in range(len(bounds) - 1):
        if value >= bounds[i] and value < bounds[i + 1]:
            return colors[i]
    return colors[-1]

result_df['color'] = result_df['sen_slope'].apply(lambda x: get_color(x, bounds, cmap.colors))

fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

gdf = gpd.read_file('study_extent.shp')

for geometry in gdf['geometry']:
    if isinstance(geometry, Polygon):
        x, y = geometry.exterior.coords.xy
        ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))
    elif isinstance(geometry, MultiPolygon):
        for subgeometry in geometry:
            x, y = subgeometry.exterior.coords.xy
            ax.add_patch(PathPatch(Path(list(zip(x, y))), fill=None, edgecolor='k', linewidth=5))

ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle='-', linewidth=2)
ax.gridlines(draw_labels=True, linestyle='--')

sc = ax.scatter(result_df['lon'], result_df['lat'], c=result_df['color'],
                marker='s', s=30, edgecolor=result_df['color'], transform=ccrs.PlateCarree())

sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

cbar = plt.colorbar(sm, orientation='horizontal', shrink=0.5, ticks=bounds, pad=0.05)
cbar.set_label('Mean Annual Recharge trend (mm)', fontsize=15)
cbar.ax.set_xticklabels(['{:.2f}'.format(b) for b in bounds])

ax.set_extent([-10, 49, 25.5, 49])


add_scalebar(ax, 500)

arrow_x, arrow_y = 0.97, .9
ax.text(arrow_x, arrow_y, u'\u25B2\nN', transform=ax.transAxes, ha='center', va='bottom', fontsize=32, fontweight='bold')

plt.title('Annual Trend in Recharge', fontsize=20, pad=20)
plt.show()
